In [28]:
import pandas as pd
import numpy as np
# import yfinance as yf


In [209]:
# carina_fp = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/raw/twitter_data/kaggle/carina_tweets_cleaned.csv'
kenny_fp = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/raw/twitter_data/kenny/kenny_stock_tweets_labelled.csv'
# big_fp = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/raw/twitter_data/big_twitter/Tweet.csv'
# jason_top = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/processed/IEEE/top.csv'
# jason_middle = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/processed/IEEE/mid.csv'
# jason_bottom = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/processed/IEEE/bottom.csv'

In [210]:
# carina_df = pd.read_csv(carina_fp)
kenny_df = pd.read_csv(kenny_fp)
# big_df = pd.read_csv(big_fp)
# jason_top_df = pd.read_csv(jason_top)
# jason_middle_df = pd.read_csv(jason_middle)
# jason_bottom_df = pd.read_csv(jason_bottom)

In [16]:
# Index(['Created_at', 'Tweet', 'Stock_ticker', 'Score', 'Source'], dtype='object')

In [17]:
jason_top_df.head(2)

,TWEET,STOCK,DATE,LAST_PRICE,1_DAY_RETURN,2_DAY_RETURN,3_DAY_RETURN,7_DAY_RETURN,PX_VOLUME,VOLATILITY_10D,VOLATILITY_30D,LSTM_POLARITY,TEXTBLOB_POLARITY
0,RT @IndiaHistorypic: 1994::Young @sundarpichai...,Google,31/01/2017,820.19,0.004438,0.030286,0.030286,0.035772,2020180.0,21.549,14.953,-1.0,0.00
1,"RT @Google: ""If you have the feeling that some...",Google,31/01/2017,820.19,0.004438,0.030286,0.030286,0.035772,2020180.0,21.549,14.953,-1.0,-0.55


In [64]:
def change_jason(df):
    df['Created_at'] = df['DATE']
    df['Company_name'] = df['STOCK']
    df['Stock_ticker'] = 99
    df['Tweet'] = df['TWEET']
    df['Score'] = df['TEXTBLOB_POLARITY']
    df['Source'] = 'Jason'
    df = df[['Created_at', 'Tweet', 'Stock_ticker', 'Company_name', 'Score', 'Source']]
    return df

In [65]:
jason_top_df = change_jason(jason_top_df)
jason_middle_df =change_jason(jason_middle_df)
jason_bottom_df = change_jason(jason_bottom_df)

In [24]:
jason_top_df.head()

,Created_at,Tweet,Stock_ticker,Company_name,Score,Source
0,31/01/2017,RT @IndiaHistorypic: 1994::Young @sundarpichai...,99,Google,0.00,Jason
1,31/01/2017,"RT @Google: ""If you have the feeling that some...",99,Google,-0.55,Jason
2,31/01/2017,"RT @IBMWatson: Welcome @Apple, #ACLU, #AAAI, @...",99,Apple,1.00,Jason
3,31/01/2017,RT @ColMorrisDavis: My family owned a @Ford de...,99,Ford,0.00,Jason
4,31/01/2017,RT @ThatBitchFromNJ: Google's sponsored ad for...,99,Google,-0.10,Jason


## Getting names for Carina

In [27]:
carina_df.head(2)

,Unnamed: 0,Created_at,Tweet,Stock_ticker,Score,Source
0,0,2020-04-09 23:59:51+00:00,@KennyDegu very very little volume. With $10T ...,SPX,-0.637271,Carina
1,1,2020-04-09 23:58:55+00:00,#ES_F achieved Target 2780 closing above 50% #...,SPX,0.919818,Carina


In [29]:
def get_company_name(ticker):
    try:
        stock = yf.Ticker(ticker)
        return stock.info.get("longName", "Unknown")
    except Exception:
        return "Unknown"


In [45]:
# carina_df.head()["Stock_ticker"].apply(get_company_name) # TOOOO FUCKING SLOOOOOOWWWWWW

In [32]:
stock_dic_fp = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/raw/twitter_data/kaggle/sp500.csv'
stock_dic_df = pd.read_csv(stock_dic_fp)

In [34]:
stock_dic_df.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,ABIOMED Inc,Health Care


In [35]:
stock_dic_df = stock_dic_df.rename(columns={"Symbol": "Stock_ticker", "Name": "Company_Name"})
carina_df = carina_df.merge(stock_dic_df, on="Stock_ticker", how="left")

In [43]:
carina_df = carina_df[~carina_df['Company_Name'].isna()]

In [51]:
carina_df = carina_df[['Created_at', 'Tweet', 'Stock_ticker', 'Score', 'Source','Company_Name', 'Sector']]

# Ssame thing for the jason dfs

In [79]:
jason_df = pd.concat([jason_top_df, jason_middle_df, jason_bottom_df], ignore_index=True)


In [84]:
jason_df['Company_Name'] = jason_df['Company_name']

In [95]:
jason_df.merge(stock_dic_df, on="Company_Name", how="left" )

358689

In [98]:
jason_df.merge(stock_dic_df, on="Company_Name", how="left" )['Stock_ticker_y'].isna().sum() / jason_df.shape[0]

0.975196567811817

In [101]:
jason_df['Company_name'].value_counts().head()

Ford        47907
Facebook    47684
Google      31657
Next        30908
Apple       29160
Name: Company_name, dtype: int64

In [107]:
jason_df.head(2)

,Created_at,Tweet,Stock_ticker,Company_name,Score,Source,Company_Name
0,31/01/2017,RT @IndiaHistorypic: 1994::Young @sundarpichai...,99,Google,0.00,Jason,Google
1,31/01/2017,"RT @Google: ""If you have the feeling that some...",99,Google,-0.55,Jason,Google


In [106]:
stock_dic_df.head(2)

,Stock_ticker,Company_Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials


In [116]:
# Using chat to create a dictionary because I can't fucking find the ticker symbols
tickers = {
    '21CF': None,  # No longer traded, was FOXA before Disney acquired it
    'ASOS': 'ASC.L',  # London Stock Exchange
    'AT&T': 'T',  # NYSE
    'Adobe': 'ADBE',  # NASDAQ
    'Allianz': 'ALV.DE',  # XETRA
    'Amazon': 'AMZN',  # NASDAQ
    'American Express': 'AXP',  # NYSE
    'Apple': 'AAPL',  # NASDAQ
    'AstraZeneca': 'AZN',  # NASDAQ, LSE: AZN.L
    'Audi': 'VOW.DE',  # Part of Volkswagen
    'Aviva': 'AV.L',  # LSE
    'BASF': 'BAS.DE',  # XETRA
    'BMW': 'BMW.DE',  # XETRA
    'BP': 'BP',  # NYSE, LSE: BP.L
    'Bank of America': 'BAC',  # NYSE
    'Bayer': 'BAYN.DE',  # XETRA
    'BlackRock': 'BLK',  # NYSE
    'Boeing': 'BA',  # NYSE
    'Burberry': 'BRBY.L',  # LSE
    'CBS': 'PARA',  # NYSE (Now part of Paramount Global)
    'CVS Health': 'CVS',  # NYSE
    'Cardinal Health': 'CAH',  # NYSE
    'Carrefour': 'CA.PA',  # Euronext Paris
    'Chevron': 'CVX',  # NYSE
    'Cisco': 'CSCO',  # NASDAQ
    'Citigroup': 'C',  # NYSE
    'CocaCola': 'KO',  # NYSE
    'Colgate': 'CL',  # NYSE
    'Comcast': 'CMCSA',  # NASDAQ
    'Costco': 'COST',  # NASDAQ
    'Danone': 'BN.PA',  # Euronext Paris
    'Deutsche Bank': 'DB',  # NYSE
    'Disney': 'DIS',  # NYSE
    'Equinor': 'EQNR',  # NYSE
    'Expedia': 'EXPE',  # NASDAQ
    'Exxon': 'XOM',  # NYSE
    'Facebook': 'META',  # NASDAQ (formerly FB)
    'FedEx': 'FDX',  # NYSE
    'Ford': 'F',  # NYSE
    'GSK': 'GSK',  # NYSE, LSE: GSK.L
    'General Electric': 'GE',  # NYSE
    'Gillette': None,  # Subsidiary of P&G
    'Goldman Sachs': 'GS',  # NYSE
    'Google': 'GOOGL',  # NASDAQ
    'Groupon': 'GRPN',  # NASDAQ
    'H&M': 'HM-B.ST',  # Stockholm Exchange
    'HP': 'HPQ',  # NYSE
    'HSBC': 'HSBC',  # NYSE, LSE: HSBA.L
    'Heineken': 'HEIA.AS',  # Euronext Amsterdam
    'Home Depot': 'HD',  # NYSE
    'Honda': 'HMC',  # NYSE
    'Hyundai': 'HYMTF',  # OTC
    'IBM': 'IBM',  # NYSE
    'Intel': 'INTC',  # NASDAQ
    'JPMorgan': 'JPM',  # NYSE
    'John Deere': 'DE',  # NYSE
    "Kellogg's": 'K',  # NYSE (now split into K and KLG)
    'Kroger': 'KR',  # NYSE
    "L'Oreal": 'OR.PA',  # Euronext Paris
    'Mastercard': 'MA',  # NYSE
    "McDonald's": 'MCD',  # NYSE
    'Microsoft': 'MSFT',  # NASDAQ
    'Morgan Stanley': 'MS',  # NYSE
    'Nestle': 'NESN.S',  # SIX Swiss Exchange
    'Netflix': 'NFLX',  # NASDAQ
    'Next': 'NXT.L',  # LSE
    'Nike': 'NKE',  # NYSE
    'Nissan': '7201.T',  # Tokyo Exchange
    'Oracle': 'ORCL',  # NYSE
    'P&G': 'PG',  # NYSE
    'PayPal': 'PYPL',  # NASDAQ
    'Pepsi': 'PEP',  # NASDAQ
    'Pfizer': 'PFE',  # NYSE
    'Reuters': None,  # Part of Thomson Reuters (NYSE: TRI)
    'Ryanair': 'RYAAY',  # NASDAQ
    'SAP': 'SAP',  # NYSE, XETRA
    'Samsung': '005930.KQ',  # Korea Exchange
    'Santander': 'SAN',  # NYSE, BMAD: SAN.MC
    'Shell': 'SHEL',  # NYSE, LSE
    'Siemens': 'SIE.DE',  # XETRA
    'Sony': 'SONY',  # NYSE
    'Starbucks': 'SBUX',  # NASDAQ
    'TMobile': 'TMUS',  # NASDAQ
    'Tesco': 'TSCO.L',  # LSE
    'Thales': 'HO.PA',  # Euronext Paris
    'Toyota': 'TM',  # NYSE
    'TripAdvisor': 'TRIP',  # NASDAQ
    'UPS': 'UPS',  # NYSE
    'Verizon': 'VZ',  # NYSE
    'Viacom': 'PARA',  # NYSE (now Paramount Global)
    'Visa': 'V',  # NYSE
    'Vodafone': 'VOD',  # NASDAQ, LSE: VOD.L
    'Volkswagen': 'VOW3.DE',  # XETRA
    'Walmart': 'WMT',  # NYSE
    'Wells Fargo': 'WFC',  # NYSE
    'Yahoo': None,  # Acquired by Verizon, no longer publicly traded
    'adidas': 'ADS.DE',  # XETRA
    'bookingcom': 'BKNG',  # NASDAQ (Booking Holdings)
    'eBay': 'EBAY',  # NASDAQ
    'easyJet': 'EZJ.L',  # LSE
    'salesforce.com': 'CRM'  # NYSE
}



In [109]:
jason_df


In [119]:
jason_df['Stock_ticker'] = jason_df['Company_Name'].map(tickers)

In [155]:
jason_df.head(1)

,Created_at,Tweet,Stock_ticker,Company_name,Score,Source,Company_Name_x,Company_Name_y,Sector
0,31/01/2017,RT @IndiaHistorypic: 1994::Young @sundarpichai...,GOOGL,Google,0.0,Jason,Google,Alphabet Inc. (Class A),Communication Services


In [122]:
sp500_tickers = [
    'T', 'ADBE', 'AMZN', 'AXP', 'AAPL', 'BAC', 'BA', 'CVS', 'CAH', 'CVX',
    'CSCO', 'C', 'KO', 'CL', 'CMCSA', 'COST', 'DIS', 'XOM', 'META', 'FDX',
    'F', 'GE', 'GS', 'GOOGL', 'GOOG', 'HD', 'IBM', 'INTC', 'JPM', 'KR',
    'MA', 'MCD', 'MSFT', 'MS', 'NFLX', 'NKE', 'ORCL', 'PG', 'PYPL', 'PEP',
    'PFE', 'SBUX', 'TMUS', 'UPS', 'VZ', 'V', 'WMT', 'WFC', 'CRM']


In [126]:
jason_df = jason_df[jason_df['Stock_ticker'].isin(sp500_tickers)]

In [128]:
jason_df = jason_df.merge(stock_dic_df, on="Stock_ticker", how="left")

## KENNY

In [211]:
kenny_df['Stock_ticker'] = kenny_df['Stock Name']


In [139]:
(stock_dic_df['Stock_ticker']=='TSLA').sum()

0

In [208]:
kenny_df

,Created_at,Tweet,Stock_ticker,Score,Source,Company_Name,Sector
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
...,...,...,...,...,...,...,...
79995,2022-05-31 11:51:02+00:00,I'm excited about $TSLA this week. Are you?,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
79996,2022-05-31 11:29:56+00:00,Tomorrow @ 11am CT we will host a YouTube even...,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
79997,2022-05-31 11:05:48+00:00,Report: $TSLA Giga Shanghai Plant Restores 70%...,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary
79998,2022-05-31 10:42:33+00:00,I didn't sell because I'm not a bull. I sold b...,TSLA,predicted_score,Kenny,Company Name,Consumer Discretionary


In [154]:
set(stock_dic_df.Sector)

{'Communication Services',
 'Consumer Discretionary',
 'Consumer Staples',
 'Energy',
 'Financials',
 'Health Care',
 'Industrials',
 'Information Technology',
 'Materials',
 'Real Estate',
 'Utilities'}

In [212]:
# using chat again
kenny_df['Sector'] = 'Consumer Discretionary'

In [213]:
kenny_df['Created_at'] = kenny_df['Date']

In [214]:
kenny_df['Source'] = 'Kenny'
kenny_df['Score'] = kenny_df['predicted_score']
kenny_df['Company_Name'] = kenny_df['Company Name']

In [215]:
# jason_df['Company_Name'] = jason_df['Company_name']
jason_df = jason_df[['Created_at', 'Tweet', 'Stock_ticker', 'Score', 'Source',
       'Company_Name', 'Sector']]

In [216]:
kenny_df = kenny_df[carina_df.columns]

In [217]:
merged_df = pd.concat([kenny_df, jason_df, carina_df], ignore_index=True)

In [220]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628827 entries, 0 to 628826
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Created_at    628827 non-null  object 
 1   Tweet         628827 non-null  object 
 2   Stock_ticker  628827 non-null  object 
 3   Score         628827 non-null  float64
 4   Source        628827 non-null  object 
 5   Company_Name  628827 non-null  object 
 6   Sector        581143 non-null  object 
dtypes: float64(1), object(6)
memory usage: 33.6+ MB


In [197]:
# merged_df['Created_at'] = pd.to_datetime(merged_df['Created_at'], errors='coerce').dt.normalize()


In [221]:
output_fp = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/processed/merged_df.csv'
merged_df

,Created_at,Tweet,Stock_ticker,Score,Source,Company_Name,Sector
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,0.549933,Kenny,"Tesla, Inc.",Consumer Discretionary
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,0.104447,Kenny,"Tesla, Inc.",Consumer Discretionary
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,0.879566,Kenny,"Tesla, Inc.",Consumer Discretionary
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,0.094424,Kenny,"Tesla, Inc.",Consumer Discretionary
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,-0.550417,Kenny,"Tesla, Inc.",Consumer Discretionary
...,...,...,...,...,...,...,...
628822,2020-04-09 11:35:12+00:00,The first informative #App on Pivot Points.\nS...,ETFC,-0.508957,Carina,E*Trade,Financials
628823,2020-04-09 11:35:08+00:00,We always try to learn from our mistakes! \n$E...,ETN,-1.070149,Carina,Eaton Corporation,Industrials
628824,2020-04-09 11:35:06+00:00,$JPM to no longer offer loans to small busines...,JPM,0.025878,Carina,JPMorgan Chase & Co.,Financials
628825,2020-04-09 11:35:03+00:00,"Brokerages Set Facebook, Inc. $FB Target Price...",FB,0.937436,Carina,Facebook Inc.,Communication Services


### BIG DF

In [181]:
big_fp = '/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/raw/twitter_data/big_twitter/big_filtered_labelled.csv'
big_df = pd.read_csv(big_fp)

In [182]:
big_df.shape

(1240000, 17)

In [184]:
big_df['Score']= big_df['predicted_score']

In [186]:
big_df['Created_at'] = pd.to_datetime(big_df['post_date'], unit='s')

In [188]:
carina_df.columns

Index(['Created_at', 'Tweet', 'Stock_ticker', 'Score', 'Source',
       'Company_Name', 'Sector'],
      dtype='object')

In [193]:
big_df['Company_Name'] = big_df['company_name']

In [201]:
big_df = big_df[['Created_at', 'Tweet', 'Stock_ticker', 'Score', 'Source',
       'Company_Name']]

In [202]:
sector_classification = {
    'Amazon.com': 'Consumer Discretionary',
    'Google Inc': 'Communication Services',
    'Microsoft': 'Information Technology',
    'Tesla Inc': 'Consumer Discretionary',
    'apple': 'Information Technology'
}

big_df['Sector'] = big_df['Company_Name'].map(sector_classification)

/tmp/ipykernel_17405/991924588.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df['Sector'] = big_df['Company_Name'].map(sector_classification)


In [224]:
big_df = big_df[merged_df.columns]

In [254]:
df = pd.concat([merged_df, big_df], ignore_index=True)

In [255]:
# set(df['Stock_ticker'])

In [256]:
# df.to_csv('/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/processed/iteration1.csv')

In [257]:
set(df['Company_Name'])

{'3M Company',
 'A.O. Smith Corp',
 'ABIOMED Inc',
 'AES Corp',
 'AFLAC Inc',
 'AMETEK Inc.',
 'ANSYS',
 'AT&T',
 'AT&T Inc.',
 'AbbVie Inc.',
 'Abbott Laboratories',
 'Accenture plc',
 'Adobe',
 'Adobe Inc.',
 'Advance Auto Parts',
 'Advanced Micro Devices Inc',
 'Agilent Technologies Inc',
 'Air Products & Chemicals Inc',
 'Alaska Air Group Inc',
 'Albemarle Corp',
 'Alexandria Real Estate Equities',
 'Alexion Pharmaceuticals',
 'Align Technology',
 'Allegion',
 'Alliant Energy Corp',
 'Allstate Corp',
 'Alphabet Inc. (Class A)',
 'Alphabet Inc. (Class C)',
 'Altria Group Inc',
 'Amazon',
 'Amazon.com',
 'Amazon.com Inc.',
 'Ameren Corp',
 'American Airlines Group',
 'American Electric Power',
 'American Express',
 'American Express Co',
 'American International Group',
 'American Water Works Company Inc',
 'Ameriprise Financial',
 'AmerisourceBergen Corp',
 'Amphenol Corp',
 'Analog Devices Inc.',
 'Anthem',
 'Aon plc',
 'Apache Corporation',
 'Apartment Investment & Management',
 '

In [249]:
# switching to chat

In [258]:
import re

# Dictionary of normalized => canonical
synonyms = {
    # AT&T
    'at&t': 'AT&T Inc.',
    'at&t inc': 'AT&T Inc.',

    # Adobe
    'adobe': 'Adobe Inc.',
    'adobe inc': 'Adobe Inc.',

    # Amazon
    'amazon': 'Amazon.com Inc.',
    'amazoncom': 'Amazon.com Inc.',
    'amazoncom inc': 'Amazon.com Inc.',

    # American Express
    'american express': 'American Express Co',
    'american express co': 'American Express Co',

    # Apple
    'apple': 'Apple Inc.',
    'apple inc': 'Apple Inc.',

    # Bank of America
    'bank of america': 'Bank of America Corp',
    'bank of america corp': 'Bank of America Corp',

    # Boeing
    'boeing': 'Boeing Company',
    'boeing company': 'Boeing Company',

    # Cardinal Health
    'cardinal health': 'Cardinal Health Inc.',
    'cardinal health inc': 'Cardinal Health Inc.',

    # Chevron
    'chevron': 'Chevron Corp.',
    'chevron corp': 'Chevron Corp.',

    # Cisco
    'cisco': 'Cisco Systems',
    'cisco systems': 'Cisco Systems',

    # Citigroup
    'citigroup': 'Citigroup Inc.',
    'citigroup inc': 'Citigroup Inc.',

    # Coca-Cola
    'cocacola': 'Coca-Cola Company',
    'coca-cola company': 'Coca-Cola Company',

    # Colgate
    'colgate': 'Colgate-Palmolive',
    'colgate-palmolive': 'Colgate-Palmolive',

    # Comcast
    'comcast': 'Comcast Corp.',
    'comcast corp': 'Comcast Corp.',

    # Costco
    'costco': 'Costco Wholesale Corp.',
    'costco wholesale corp': 'Costco Wholesale Corp.',

    # Disney
    'disney': 'The Walt Disney Company',
    'the walt disney company': 'The Walt Disney Company',

    # Exxon
    'exxon': 'Exxon Mobil Corp.',
    'exxon mobil corp': 'Exxon Mobil Corp.',

    # Facebook (aka Meta)
    'facebook': 'Facebook Inc.',
    'facebook inc': 'Facebook Inc.',

    # FedEx
    'fedex': 'FedEx Corporation',
    'fedex corporation': 'FedEx Corporation',

    # Ford
    'ford': 'Ford Motor Company',
    'ford motor company': 'Ford Motor Company',

    # Goldman Sachs
    'goldman sachs': 'Goldman Sachs Group',
    'goldman sachs group': 'Goldman Sachs Group',

    # Google / Alphabet – unify under Alphabet Inc. 
    'google': 'Alphabet Inc.',
    'google inc': 'Alphabet Inc.',
    'alphabet inc class a': 'Alphabet Inc.',
    'alphabet inc class c': 'Alphabet Inc.',
    # The next two lines handle parentheses in "Alphabet Inc. (Class A)" or (Class C)
    'alphabet inc (class a)': 'Alphabet Inc.',
    'alphabet inc (class c)': 'Alphabet Inc.',

    # IBM
    'ibm': 'International Business Machines',
    'international business machines': 'International Business Machines',

    # Intel
    'intel': 'Intel Corp.',
    'intel corp': 'Intel Corp.',

    # JPMorgan
    'jpmorgan': 'JPMorgan Chase & Co.',
    'jpmorgan chase & co': 'JPMorgan Chase & Co.',

    # Kroger
    'kroger': 'Kroger Co.',
    'kroger co': 'Kroger Co.',

    # Mastercard
    'mastercard': 'Mastercard Inc.',
    'mastercard inc': 'Mastercard Inc.',

    # McDonald's
    "mcdonald's": "McDonald's Corp.",
    "mcdonald's corp": "McDonald's Corp.",

    # Microsoft
    'microsoft': 'Microsoft Corp.',
    'microsoft corp': 'Microsoft Corp.',

    # Netflix
    'netflix': 'Netflix Inc.',
    'netflix inc': 'Netflix Inc.',

    # Nike
    'nike': 'Nike Inc.',
    'nike inc': 'Nike Inc.',

    # Oracle
    'oracle': 'Oracle Corp.',
    'oracle corp': 'Oracle Corp.',

    # Procter & Gamble
    'p&g': 'Procter & Gamble',
    'procter & gamble': 'Procter & Gamble',

    # Pepsi
    'pepsi': 'PepsiCo Inc.',
    'pepsico inc': 'PepsiCo Inc.',

    # Pfizer
    'pfizer': 'Pfizer Inc.',
    'pfizer inc': 'Pfizer Inc.',

    # Salesforce
    'salesforcecom': 'salesforce.com',  # or "Salesforce, Inc." if you prefer
    # (choose one form; “Salesforce, Inc.” is the modern legal name)
    'salesforce.com': 'salesforce.com',

    # Starbucks
    'starbucks': 'Starbucks Corp.',
    'starbucks corp': 'Starbucks Corp.',

    # T-Mobile
    'tmobile': 'T-Mobile US',

    # Tesla
    'tesla inc': 'Tesla, Inc.',
    'tesla, inc': 'Tesla, Inc.',
    'tesla': 'Tesla, Inc.',

    # UPS
    'ups': 'United Parcel Service',
    'united parcel service': 'United Parcel Service',

    # Verizon
    'verizon': 'Verizon Communications',
    'verizon communications': 'Verizon Communications',

    # Visa
    'visa': 'Visa Inc.',
    'visa inc': 'Visa Inc.'
}

def consolidate_company_name(company_name: str) -> str:
    """
    Return a canonical version of the company_name if it matches
    a known synonym, otherwise return the name as-is.
    """
    # Convert to lowercase, remove commas and periods, etc. (except & or apostrophes).
    normalized = re.sub(r'[.,]', '', company_name.strip().lower())

    # Look up in synonyms dictionary
    return synonyms.get(normalized, company_name)

# ---------------------------------------------------------------------------------------------
# EXAMPLE USAGE:
df['Consolidated_Company_Name'] = df['Company_Name'].apply(consolidate_company_name)


In [259]:
df

,Created_at,Tweet,Stock_ticker,Score,Source,Company_Name,Sector,Consolidated_Company_Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,0.549933,Kenny,"Tesla, Inc.",Consumer Discretionary,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,0.104447,Kenny,"Tesla, Inc.",Consumer Discretionary,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,0.879566,Kenny,"Tesla, Inc.",Consumer Discretionary,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,0.094424,Kenny,"Tesla, Inc.",Consumer Discretionary,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,-0.550417,Kenny,"Tesla, Inc.",Consumer Discretionary,"Tesla, Inc."
...,...,...,...,...,...,...,...,...
1868822,2018-09-01 14:11:07,"As Apple INC $AAPL Share Value Rose, Waddell &...",AAPL,0.934082,Ginger,apple,Information Technology,Apple Inc.
1868823,2018-09-01 14:11:44,Contrasting Alphabet Inc Class A $GOOGL and Sy...,GOOGL,-1.074219,Ginger,Google Inc,Communication Services,Alphabet Inc.
1868824,2018-09-01 14:12:07,Nanoleaf's touch-sensitive Canvas light panels...,AAPL,0.943848,Ginger,apple,Information Technology,Apple Inc.
1868825,2018-09-01 14:13:16,Financial Review: Synacor $SYNC & Alphabet Inc...,GOOGL,0.827637,Ginger,Google Inc,Communication Services,Alphabet Inc.


In [263]:
len(set(df['Consolidated_Company_Name']))

298

In [261]:

ticker_synonyms = {
    'GOOG': 'GOOGL',    # unify GOOG to GOOGL
    'FB': 'META'        # unify FB to META
    # etc. add more if needed
}

def consolidate_ticker(ticker: str) -> str:
    """
    Return a canonical ticker if it matches a known duplicate,
    otherwise return the original ticker.
    """
    return ticker_synonyms.get(ticker, ticker)

# Create a new column with consolidated tickers
df['Consolidated_Ticker'] = df['Stock_ticker'].apply(consolidate_ticker)



In [268]:
company_counts = df['Consolidated_Company_Name'].value_counts()
# company_counts

Tesla, Inc.             430572
Apple Inc.              389896
Amazon.com Inc.         288154
Alphabet Inc.           223877
Microsoft Corp.         138347
                         ...  
Juniper Networks            92
Grainger (W.W.) Inc.        92
Sealed Air                  92
Zoetis                      92
CVS Health                  10
Name: Consolidated_Company_Name, Length: 298, dtype: int64

In [274]:
filtered_companies = company_counts[company_counts > 500].index
df_filtered = df[df['Consolidated_Company_Name'].isin(filtered_companies)]
len(set(df_filtered['Consolidated_Company_Name']))

57

In [277]:
set(df_filtered['Consolidated_Company_Name'])

{'AT&T Inc.',
 'Adobe Inc.',
 'Advanced Micro Devices Inc',
 'Air Products & Chemicals Inc',
 'Alphabet Inc.',
 'Amazon.com Inc.',
 'Apple Inc.',
 'Bank of America Corp',
 'Boeing Company',
 'Broadcom Inc.',
 'Carnival Corp.',
 'Caterpillar Inc.',
 'Chevron Corp.',
 'Cisco Systems',
 'Citigroup Inc.',
 'Coca-Cola Company',
 'Comcast Corp.',
 'Costco Wholesale Corp.',
 'Dow Inc.',
 'Edwards Lifesciences',
 'Eversource Energy',
 'Exxon Mobil Corp.',
 'Facebook Inc.',
 'FedEx Corporation',
 'Ford Motor Company',
 'Goldman Sachs Group',
 'Home Depot',
 'Intel Corp.',
 'International Business Machines',
 'JPMorgan Chase & Co.',
 'Johnson & Johnson',
 'Kroger Co.',
 'Mastercard Inc.',
 "McDonald's Corp.",
 'Microsoft Corp.',
 'Morgan Stanley',
 'Netflix Inc.',
 'Newmont Corporation',
 'Nike Inc.',
 'Nvidia Corporation',
 'Oracle Corp.',
 'PayPal',
 'PepsiCo Inc.',
 'Pfizer Inc.',
 'Procter & Gamble',
 'Starbucks Corp.',
 'T-Mobile US',
 'Tesla, Inc.',
 'The Bank of New York Mellon',
 'The Wa

In [278]:
gics_sub_industry_map = {
    "AT&T Inc.": "Integrated Telecommunication Services",
    "Adobe Inc.": "Application Software",
    "Advanced Micro Devices Inc": "Semiconductors",
    "Air Products & Chemicals Inc": "Industrial Gases",
    "Alphabet Inc.": "Interactive Media & Services",
    "Amazon.com Inc.": "Internet & Direct Marketing Retail",
    "Apple Inc.": "Technology Hardware, Storage & Peripherals",
    "Bank of America Corp": "Diversified Banks",
    "Boeing Company": "Aerospace & Defense",
    "Broadcom Inc.": "Semiconductors",
    "Carnival Corp.": "Hotels, Resorts & Cruise Lines",
    "Caterpillar Inc.": "Construction Machinery & Heavy Trucks",
    "Chevron Corp.": "Integrated Oil & Gas",
    "Cisco Systems": "Communications Equipment",
    "Citigroup Inc.": "Diversified Banks",
    "Coca-Cola Company": "Soft Drinks",
    "Comcast Corp.": "Cable & Satellite",
    "Costco Wholesale Corp.": "Hypermarkets & Super Centers",
    "Dow Inc.": "Commodity Chemicals",
    "Edwards Lifesciences": "Health Care Equipment",
    "Eversource Energy": "Electric Utilities",
    "Exxon Mobil Corp.": "Integrated Oil & Gas",
    "Facebook Inc.": "Interactive Media & Services",
    "FedEx Corporation": "Air Freight & Logistics",
    "Ford Motor Company": "Automobile Manufacturers",
    "Goldman Sachs Group": "Investment Banking & Brokerage",
    "Home Depot": "Home Improvement Retail",
    "Intel Corp.": "Semiconductors",
    "International Business Machines": "IT Consulting & Other Services",
    "JPMorgan Chase & Co.": "Diversified Banks",
    "Johnson & Johnson": "Pharmaceuticals",
    "Kroger Co.": "Food Retail",
    "Mastercard Inc.": "Data Processing & Outsourced Services",
    "McDonald's Corp.": "Restaurants",
    "Microsoft Corp.": "Systems Software",
    "Morgan Stanley": "Investment Banking & Brokerage",
    "Netflix Inc.": "Movies & Entertainment",
    "Newmont Corporation": "Gold",
    "Nike Inc.": "Footwear",
    "Nvidia Corporation": "Semiconductors",
    "Oracle Corp.": "Systems Software",
    "PayPal": "Data Processing & Outsourced Services",
    "PepsiCo Inc.": "Soft Drinks",
    "Pfizer Inc.": "Pharmaceuticals",
    "Procter & Gamble": "Household Products",
    "Starbucks Corp.": "Restaurants",
    "T-Mobile US": "Wireless Telecommunication Services",
    "Tesla, Inc.": "Automobile Manufacturers",
    "The Bank of New York Mellon": "Asset Management & Custody Banks",
    "The Walt Disney Company": "Movies & Entertainment",
    "Twitter Inc.": "Interactive Media & Services",
    "United Health Group Inc.": "Managed Health Care",
    "United Parcel Service": "Air Freight & Logistics",
    "Verizon Communications": "Integrated Telecommunication Services",
    "Visa Inc.": "Data Processing & Outsourced Services",
    "Walmart": "Hypermarkets & Super Centers",
    "Wells Fargo": "Diversified Banks"
}


In [283]:
df_filtered["GICS_Sub_Industry"] = df_filtered["Consolidated_Company_Name"].map(gics_sub_industry_map).fillna("Unknown")


/tmp/ipykernel_17405/2377444709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["GICS_Sub_Industry"] = df_filtered["Consolidated_Company_Name"].map(gics_sub_industry_map).fillna("Unknown")


In [286]:
df_filtered.to_csv('/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/processed/iteration2.csv')

In [288]:
df_filtered["Created_at"] = pd.to_datetime(df_filtered["Created_at"])

/tmp/ipykernel_17405/175842938.py:1: UserWarning: Parsing '31/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_filtered["Created_at"] = pd.to_datetime(df_filtered["Created_at"])
/tmp/ipykernel_17405/175842938.py:1: UserWarning: Parsing '28/02/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_filtered["Created_at"] = pd.to_datetime(df_filtered["Created_at"])
/tmp/ipykernel_17405/175842938.py:1: UserWarning: Parsing '31/03/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_filtered["Created_at"] = pd.to_datetime(df_filtered["Created_at"])
/tmp/ipykernel_17405/175842938.py:1: UserWarning: Parsing '28/04/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_filtered["Created_at"] = pd.to_datetime(df_filtered["Created_at"])
/tmp/ipykernel_17405/175842938.p

In [290]:
df_filtered.to_csv('/home/ginger/code/gderiddershanghai/DVA_Team_173/data_full/processed/iteration3.csv')

In [302]:
l = ['Chevron Corp.',
 'Cisco Systems',
 'Citigroup Inc.',
 'Coca-Cola Company',
 'Comcast Corp.',
 'Costco Wholesale Corp.',
 'Dow Inc.',
 'Edwards Lifesciences',
 'Eversource Energy',
 'Exxon Mobil Corp.',
 'Facebook Inc.',
 'FedEx Corporation',
 # 'Ford Motor Company',
 'Goldman Sachs Group']

In [306]:
df_filtered.sample(3)

,Created_at,Tweet,Stock_ticker,Score,Source,Company_Name,Sector,Consolidated_Company_Name,Consolidated_Ticker,GICS_Sub_Industry
244502,2018-01-10 00:00:00,"RT @itsKammy_G: ""The childhood of many will be...",DIS,0.600000,Jason,Disney,Communication Services,The Walt Disney Company,DIS,Movies & Entertainment
1539500,2018-05-01 23:32:32,Shortfall in battery metals to slow electric v...,TSLA,0.953613,Ginger,Tesla Inc,Consumer Discretionary,"Tesla, Inc.",TSLA,Automobile Manufacturers
482424,2020-04-09 12:26:31+00:00,$JNJ swiftly 3 weeks higher took the month up ...,JNJ,0.958321,Carina,Johnson & Johnson,Health Care,Johnson & Johnson,JNJ,Pharmaceuticals


In [307]:
len(set(df_filtered['Consolidated_Company_Name']))

57